In [2]:
import opendatasets as od
od.download('https://www.kaggle.com/datasets/tongtrantiendung/rl-sentence-compression', force=True)

/Users/achintya/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


100%|██████████| 423M/423M [00:41<00:00, 10.7MB/s] 


In [6]:
import pandas as pd

data = pd.read_json('rl-sentence-compression/rl-sentence-compression/rl-sentence-compression/data/train-data/gigaword/train.jsonl', lines=True)

In [7]:
data = data[:1000]

In [8]:
data = data.drop_duplicates()

data = data.dropna()

data['text'] = data['text'].str.lower()

data['text'] = data['text'].str.replace('[^\w\s]', '')

data['text'] = data['text'].str.split()

stop_words = ['the', 'a', 'an', 'and', 'or', 'in', 'on', 'at', 'to']
data['text'] = data['text'].apply(lambda x: [word for word in x if word not in stop_words])

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
data['text'] = data['text'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

data['text'] = data['text'].apply(lambda x: [word for word in x if not word.isdigit()])

data['text'] = data['text'].apply(lambda x: ' '.join(x))

data = data.reset_index(drop=True)


In [18]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, GRU, Dense
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
import numpy as np
X_train, X_val, y_train, y_val = train_test_split(data['text'], data['summary'], test_size=0.2, random_state=42)

max_vocab = 10000
max_len = 100
vectorizer = TextVectorization(max_tokens=max_vocab, output_sequence_length=max_len)
vectorizer.adapt(X_train.values)

X_train_seq = vectorizer(X_train.values)
X_val_seq = vectorizer(X_val.values)

vectorizer = TextVectorization(max_tokens=max_vocab, output_sequence_length=max_len)
vectorizer.adapt(y_train.values)

y_train_seq = vectorizer(y_train.values)
y_val_seq = vectorizer(y_val.values)

model = Sequential()
model.add(Embedding(max_vocab, 128))
model.add(SimpleRNN(128, return_sequences=True))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dense(max_vocab, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

model.fit(X_train_seq, y_train_seq, validation_data=(X_val_seq, y_val_seq), epochs=10, batch_size=128)

Epoch 1/10
7/7 [==============================] - 5s 633ms/step - loss: 8.6874 - accuracy: 0.6197 - val_loss: 7.6686 - val_accuracy: 0.9219
Epoch 2/10
7/7 [==============================] - 5s 705ms/step - loss: 7.0138 - accuracy: 0.9236 - val_loss: 5.9451 - val_accuracy: 0.9219
Epoch 3/10
7/7 [==============================] - 5s 740ms/step - loss: 5.2769 - accuracy: 0.9236 - val_loss: 4.2016 - val_accuracy: 0.9219
Epoch 4/10
7/7 [==============================] - 4s 595ms/step - loss: 3.5486 - accuracy: 0.9236 - val_loss: 2.5506 - val_accuracy: 0.9219
Epoch 5/10
7/7 [==============================] - 4s 634ms/step - loss: 2.0238 - accuracy: 0.9236 - val_loss: 1.3704 - val_accuracy: 0.9219
Epoch 6/10
7/7 [==============================] - 5s 664ms/step - loss: 1.1371 - accuracy: 0.9236 - val_loss: 0.9440 - val_accuracy: 0.9219
Epoch 7/10
7/7 [==============================] - 4s 578ms/step - loss: 0.8728 - accuracy: 0.9236 - val_loss: 0.8544 - val_accuracy: 0.9219
Epoch 8/10
7/7 [====

In [24]:
model_lstm = Sequential()
model_lstm.add(Embedding(max_vocab, 128))
model_lstm.add(LSTM(128, return_sequences=True))
model_lstm.add(LSTM(128, return_sequences=True))
model_lstm.add(Dense(max_vocab, activation='softmax'))

model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

model_lstm.fit(X_train_seq, y_train_seq, validation_data=(X_val_seq, y_val_seq), epochs=10, batch_size=128)

Epoch 1/10
7/7 [==============================] - 8s 1s/step - loss: 9.1570 - accuracy: 0.7731 - val_loss: 8.7865 - val_accuracy: 0.9219
Epoch 2/10
7/7 [==============================] - 7s 1s/step - loss: 8.0726 - accuracy: 0.9236 - val_loss: 6.7906 - val_accuracy: 0.9219
Epoch 3/10
7/7 [==============================] - 6s 858ms/step - loss: 5.9113 - accuracy: 0.9236 - val_loss: 4.4795 - val_accuracy: 0.9219
Epoch 4/10
7/7 [==============================] - 6s 904ms/step - loss: 3.6818 - accuracy: 0.9236 - val_loss: 2.4988 - val_accuracy: 0.9219
Epoch 5/10
7/7 [==============================] - 6s 903ms/step - loss: 1.9223 - accuracy: 0.9236 - val_loss: 1.2407 - val_accuracy: 0.9219
Epoch 6/10
7/7 [==============================] - 7s 1s/step - loss: 1.0312 - accuracy: 0.9236 - val_loss: 0.8748 - val_accuracy: 0.9219
Epoch 7/10
7/7 [==============================] - 5s 742ms/step - loss: 0.8158 - accuracy: 0.9236 - val_loss: 0.8088 - val_accuracy: 0.9219
Epoch 8/10
7/7 [=============

In [25]:
model_gru = Sequential()
model_gru.add(Embedding(max_vocab, 128))
model_gru.add(GRU(128, return_sequences=True))
model_gru.add(GRU(128, return_sequences=True))
model_gru.add(Dense(max_vocab, activation='softmax'))

model_gru.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

model_gru.fit(X_train_seq, y_train_seq, validation_data=(X_val_seq, y_val_seq), epochs=10, batch_size=128)

Epoch 1/10
7/7 [==============================] - 7s 832ms/step - loss: 9.1593 - accuracy: 0.7725 - val_loss: 8.9605 - val_accuracy: 0.9219
Epoch 2/10
7/7 [==============================] - 5s 738ms/step - loss: 8.3788 - accuracy: 0.9236 - val_loss: 6.5457 - val_accuracy: 0.9219
Epoch 3/10
7/7 [==============================] - 5s 756ms/step - loss: 5.4208 - accuracy: 0.9236 - val_loss: 4.0450 - val_accuracy: 0.9219
Epoch 4/10
7/7 [==============================] - 5s 701ms/step - loss: 3.3440 - accuracy: 0.9236 - val_loss: 2.2781 - val_accuracy: 0.9219
Epoch 5/10
7/7 [==============================] - 5s 694ms/step - loss: 1.7583 - accuracy: 0.9236 - val_loss: 1.1686 - val_accuracy: 0.9219
Epoch 6/10
7/7 [==============================] - 5s 753ms/step - loss: 0.9934 - accuracy: 0.9236 - val_loss: 0.8750 - val_accuracy: 0.9219
Epoch 7/10
7/7 [==============================] - 6s 874ms/step - loss: 0.8215 - accuracy: 0.9236 - val_loss: 0.8231 - val_accuracy: 0.9219
Epoch 8/10
7/7 [====